In [ ]:
! pip install langchain==0.1.14 openai==1.14.3 langchain-openai==0.1.1
! pip install pypdf
! pip install langchain-chroma

In [ ]:
# 독서모임 플로우 수정 버전
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import time


class RAGBookClubGPT:
    def __init__(self, pdf_path, api_key):
        """
        RAG 기반 독서 모임 AI 초기화
        pdf_path: 독서 모임 관련 PDF 문서 경로
        api_key: OpenAI API 키
        """

        # 데이터 로드
        loader = PyPDFLoader(pdf_path)
        self.docs = loader.load()

        # 벡터 임베딩 생성 및 데이터베이스 구축
        embeddings_model = OpenAIEmbeddings(api_key=api_key)
        self.db = Chroma.from_documents(self.docs, embeddings_model)

        # 데이터 검색기 및 QA 체인 초기화
        self.retriever = self.db.as_retriever()
        self.llm = ChatOpenAI(model_name="gpt-4", api_key=api_key)
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever
        )

    def generate_topics(self, user_responses):
        """
        RAG 기반으로 발제문 생성
        user_responses: 참가자 감상문
        """

        system_message = "Generate three non-political, open-ended discussion topics in Korean based on this book. The topics must end with a sentence."
        user_message = "\n".join(user_responses)

        # RAG 기반 문맥 검색 및 발제문 생성
        context_docs = self.retriever.get_relevant_documents("발제문 생성에 필요한 주요 내용")
        context_text = "\n".join(doc.page_content for doc in context_docs)

        prompt = f"{system_message}\n\nPDF 문서 내용:\n{context_text}\n\n참가자 감상문:\n{user_message}"
        response = self.llm.predict(prompt)

        # 발제문 나누기 및 공백 제거
        topics = [topic.strip() for topic in response.split("\n") if topic.strip()]
        if len(topics) < 3:
            print("⚠️ 발제문 생성에 실패했습니다. 출력을 확인하세요.")
            print(response)
            return []

        print("선정된 발제문:")
        for topic in topics:
            print(f"- {topic}")

        return topics

    def collect_responses(self, topic):
        """
        주어진 주제에 대해 참가자들의 의견을 수집하고, 욕설 및 비방 내용을 필터링하는 메서드
        """

        print(f'\n주제: {topic}')
        print('이 주제에 대해 의견을 작성해 주세요.')

        responses = []
        while True:
            user_input = input("사용자 입력 (형식: 사용자명: 의견, 종료 시 '종료' 입력): ").strip()
            if user_input.lower() == "종료":
                break

            try:
                user, response = user_input.split(":", 1)
                responses.append(f"{user.strip()}: {response.strip()}")
            except ValueError:
                print("잘못된 입력 형식입니다. '사용자명: 의견' 형식으로 입력해주세요.")

        filtered_responses = self.filter_responses(responses)
        print('필터링된 의견:')
        for response in filtered_responses:
            print(response)

        return filtered_responses

    def filter_responses(self, responses):
        """
        참가자들의 응답에서 욕설이나 비방이 포함된 내용을 필터링하는 메서드.
        """

        bad_words = ['욕설', '비방']
        filtered = [response for response in responses if not any(bad_word in response for bad_word in bad_words)]
        return filtered


    def facilitate_free_discussion(self, opinion, time_limit=10):
        """
        특정 의견에 대한 자유 토의를 제한 시간 동안 진행
        opinion: 논의 중심이 될 사용자 의견
        time_limit: 제한 시간(초 단위)
        """
        # 테스트를 위해 test_limit = 10으로 설정. 원래는 180
        print(f"💡이 의견에 대해 3분간 자유롭게 이야기를 나눠주세요")
        start_time = time.time()

        while True:
            elapsed_time = time.time() - start_time
            remaining_time = time_limit - elapsed_time

            if remaining_time <= 0:
                print("\n⏳ 자유토의 시간이 종료되었습니다.")
                break

            print(f"⏱️ 남은 시간: {int(remaining_time)}초", end="\r")
            time.sleep(1)

        print("다음 의견으로 넘어갑니다.")

    def summarize_meeting(self, topics, all_responses):
        """
        RAG 기반으로 모임 내용을 요약하는 메서드
        topics: 논의된 주제들
        all_responses: 참가자들의 모든 응답
        """

        print('\n모임 내용을 요약합니다...')
        summary_prompt = "다음은 독서 모임에서 논의된 내용입니다. 각 주제에 대해 요약을 작성해줘.:\n"

        for i, topic in enumerate(topics):
            summary_prompt += f"주제 {i + 1}: {topic}\n응답:\n" + "\n".join(all_responses[i]) + "\n"

        response = self.llm.predict(summary_prompt)
        summary = response.strip()

        print("모임 요약:")
        print(summary)

    def run_meeting(self, user_responses):
        """
        전체 독서 모임 흐름 실행
        """

        topics = self.generate_topics(user_responses)
        if not topics:
            print("모임을 시작할 수 없습니다. 발제문을 생성하지 못했습니다.")
            return

        all_responses = []
        for topic in topics:
            responses = self.collect_responses(topic)
            all_responses.append(responses)

            for opinion in responses:
                print(f"현재 의견: {opinion}")
                self.facilitate_free_discussion(opinion, time_limit=10) # 테스트를 위해 10초로 설정

        self.summarize_meeting(topics, all_responses)



In [ ]:
# PDF 경로와 OpenAI API 키 입력
pdf_path = "/content/1984.pdf"  # PDF 파일 경로
api_key = "##API key 입력##"  # OpenAI API 키

# RAGBookClubGPT 인스턴스 생성
rag_book_club = RAGBookClubGPT(pdf_path, api_key)

# 사용자 감상문 입력 (예시)
user_responses = [
  "사용자1: 1984는 인간의 자유와 권리가 어떻게 억압될 수 있는지를 극명하게 보여주는 작품입니다. 소설 속에서 '빅 브라더'라는 상징적인 존재는 모든 시민을 감시하고 통제하며, 개인의 사생활을 철저히 침해합니다. 이러한 설정은 현대 사회에서도 점점 강화되고 있는 감시 기술과 프라이버시 침해 문제를 떠올리게 합니다. 특히, 정부나 기업이 개인의 데이터를 수집하고 이를 활용하는 방식은 소설 속 감시 체제와 유사한 점이 많습니다. 오웰은 이러한 미래를 경고하며, 독자로 하여금 자유와 권리를 지키기 위해 경각심을 갖도록 촉구합니다. 이 작품은 기술 발전이 가져올 수 있는 부정적인 측면을 경계하게 만들며, 개인의 자유가 얼마나 소중한지를 일깨워 줍니다.",
  "사용자2: 조지 오웰의 1984는 전체주의 체제의 위험성을 경고하는 강력한 메시지를 담고 있습니다. 소설 속 당은 과거를 조작하고, 언어를 통제하며, 개인의 사고까지 지배하려 합니다. 이는 비판적 사고가 억압될 때 사회가 얼마나 쉽게 무너질 수 있는지를 보여줍니다. 주인공 윈스턴 스미스는 체제에 저항하려 하지만, 결국 당의 압도적인 힘 앞에서 무력함을 느끼게 됩니다. 이는 독재 체제 하에서 인간성이 어떻게 파괴될 수 있는지를 생생하게 전달합니다. 오웰은 독자들에게 비판적 사고와 자유로운 의사 표현이 얼마나 중요한지를 상기시키며, 이러한 가치들이 억압될 때 사회가 어떤 방향으로 나아갈 수 있는지를 경고합니다.",
  "사용자3: 오웰의 1984는 인간 본성에 대한 깊은 통찰을 제공합니다. 윈스턴 스미스는 사랑과 자유를 추구하지만, 결국 체제에 굴복하게 됩니다. 이는 인간의 본성과 욕망이 어떻게 억압될 수 있는지를 보여줍니다. 소설은 독자로 하여금 개인의 자유와 인간성을 지키기 위해 끊임없이 노력해야 한다는 메시지를 전달합니다. 특히, 사랑과 우정 같은 인간적 유대가 어떻게 체제에 의해 파괴될 수 있는지를 통해 인간성 상실의 위험성을 경고합니다. 이는 오늘날에도 여전히 중요한 교훈으로 남아 있으며, 개인이 자신의 정체성과 가치를 지키기 위해 어떤 노력을 해야 하는지를 고민하게 만듭니다.",
  "사용자4: 흥미로웠습니다. 디스토피아 소설의 원조답게 재밌게 잘 읽었습니다. 현재와도 맞닿아 있는 부분이 많아서 작가의 통찰력이 대단하다는 생각이 들었습니다."
  ]

# 독서 모임 실행
rag_book_club.run_meeting(user_responses)


선정된 발제문:
- 1. "1984"에서 조지 오웰은 개인의 사생활이 철저히 침해되는 상황을 묘사하고 있습니다. 이와 같은 상황이 현대 사회에서도 벌어지고 있다고 생각하시나요? 그렇다면 그 예는 무엇인가요?
- 2. "1984"는 전체주의 체제에서 비판적 사고가 억압될 때 사회가 어떻게 쉽게 무너질 수 있는지 보여줍니다. 현대 사회에서 비판적 사고를 억압하는 경우를 본 적이 있나요? 만약 있다면 그 사례는 무엇인가요?
- 3. 소설에서는 사랑과 우정 같은 인간적 유대가 체제에 의해 파괴되는 모습을 보여줍니다. 현실에서는 어떤 경우에 이런 상황이 벌어질 수 있을까요? 이에 대한 여러분의 생각은 어떠한가요?

주제: 1. "1984"에서 조지 오웰은 개인의 사생활이 철저히 침해되는 상황을 묘사하고 있습니다. 이와 같은 상황이 현대 사회에서도 벌어지고 있다고 생각하시나요? 그렇다면 그 예는 무엇인가요?
이 주제에 대해 의견을 작성해 주세요.
사용자 입력 (형식: 사용자명: 의견, 종료 시 '종료' 입력): 1984에서 빅 브라더는 텔레스크린을 통해 시민들을 감시합니다. 이는 현대 사회에서 스마트폰, CCTV, 인터넷 기기들이 개인의 활동을 추적하는 방식과 유사합니다. 예를 들어, 스마트폰의 위치 추적 기능이나 인터넷 사용 기록은 개인의 일상생활을 감시할 수 있는 도구로 활용됩니다15. 이러한 기술 발전은 편리함을 제공하는 동시에 사생활 침해의 위험을 증가시킵니다.
잘못된 입력 형식입니다. '사용자명: 의견' 형식으로 입력해주세요.
사용자 입력 (형식: 사용자명: 의견, 종료 시 '종료' 입력): 사용자1: 1984에서 빅 브라더는 텔레스크린을 통해 시민들을 감시합니다. 이는 현대 사회에서 스마트폰, CCTV, 인터넷 기기들이 개인의 활동을 추적하는 방식과 유사합니다. 예를 들어, 스마트폰의 위치 추적 기능이나 인터넷 사용 기록은 개인의 일상생활을 감시할 수 있는 도구로 활용됩니다15. 이러한 기술 발전은 편리함을 제공하는 동시에 사생활 침해의 위험을 증가시킵니다.
